# **Convolutional Neural Network**
*   Implementation of a convolutional neural network classifier supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a convolutional neural network classifier supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementationa are based off of TensorFlow's very own example: [TensorFlow CNN Tutorial](https://www.tensorflow.org/tutorials/images/cnn)

In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [2]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 2
# Values are specific to the CIFAR10 dataset
INPUT_DIM = 32
INPUT_CHANNELS = 3
NUM_CLASSES = 10

In [3]:
def build_and_train(X_train, y_train, precision='single'):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(INPUT_DIM, INPUT_DIM, INPUT_CHANNELS), dtype=dtype),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', dtype=dtype),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(dtype=dtype),
        layers.Dense(64, activation='relu', dtype=dtype),
        layers.Dense(NUM_CLASSES, activation='softmax', dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [4]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(INPUT_DIM, INPUT_DIM, INPUT_CHANNELS)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [5]:
# Load dataset and split into train and test sets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

# Scale values before feeding into neural net
X_train = X_train / 255.0
X_test = X_test / 255.0

170498071/170498071 [==============================] - 4s 0us/step


In [6]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1563/1563 [==============================] - 12s 5ms/step - loss: 1.5065 - accuracy: 0.4528
Epoch 2/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.1358 - accuracy: 0.5975
Epoch 3/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.9887 - accuracy: 0.6528
Epoch 4/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8970 - accuracy: 0.6886
Epoch 5/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8251 - accuracy: 0.7093
Epoch 1/5
1563/1563 [==============================] - 13s 6ms/step - loss: 1.5284 - accuracy: 0.4429
Epoch 2/5
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1512 - accuracy: 0.5903
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0088 - accuracy: 0.6442
Epoch 4/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9085 - accuracy: 0.6823
Epoch 5/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8327 - accuracy: 0.710

In [7]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 16s 8ms/step - loss: 1.5370 - accuracy: 0.4379
Epoch 2/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.1466 - accuracy: 0.5959
Epoch 3/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.9908 - accuracy: 0.6526
Epoch 4/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.8934 - accuracy: 0.6860
Epoch 5/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.8238 - accuracy: 0.7120
Epoch 1/5
1563/1563 [==============================] - 15s 8ms/step - loss: 1.5142 - accuracy: 0.4502
Epoch 2/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.1539 - accuracy: 0.5923
Epoch 3/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.0075 - accuracy: 0.6454
Epoch 4/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.9104 - accuracy: 0.6802
Epoch 5/5
1563/1563 [==============================] - 12s 8ms/step - loss: 0.8396

In [8]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.5211 - accuracy: 0.4444
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1408 - accuracy: 0.5966
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9774 - accuracy: 0.6585
Epoch 4/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8906 - accuracy: 0.6897
Epoch 5/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8144 - accuracy: 0.7132
Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.4924 - accuracy: 0.4553
Epoch 2/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.1261 - accuracy: 0.6023
Epoch 3/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.9776 - accuracy: 0.6552
Epoch 4/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8800 - accuracy: 0.6903
Epoch 5/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8032 - accurac

In [9]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: nan - accuracy: 0.0999
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 2/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
313/313 - 1s - loss:

In [12]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 10s 5ms/step - loss: 1.5039 - accuracy: 0.4495
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1407 - accuracy: 0.5971
Epoch 3/5
1563/1563 [==============================] - 9s 5ms/step - loss: 0.9829 - accuracy: 0.6546
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8857 - accuracy: 0.6920
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8144 - accuracy: 0.7153
Epoch 1/5
1563/1563 [==============================] - 11s 6ms/step - loss: 1.5389 - accuracy: 0.4406
Epoch 2/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.1712 - accuracy: 0.5854
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0250 - accuracy: 0.6390
Epoch 4/5
1563/1563 [==============================] - 9s 5ms/step - loss: 0.9297 - accuracy: 0.6723
Epoch 5/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8632 - accur

In [13]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_single)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 84.81147956848145 seconds
Average training time in single precision: 40.99379587173462 seconds
Average training time in half precision: 39.33662962913513 seconds
Average training time in mixed half precision: 85.53620910644531 seconds
-------------
Accuracy with double precision: 0.6931999921798706
Accuracy with single precision: 0.6851000189781189
Accuracy with half precision: 0.6851000189781189
Accuracy with mixed half precision: 0.6894000172615051
